<h1>Chapter 08. Type Annotations in Functions.</h1>

Type annotations in functions in Python allow you to specify the types of parameters and return values. They provide clarity about the expected types of input and output, making code easier to understand and maintain. Type annotations are optional but can be beneficial for documentation, static analysis, and catching type-related bugs early.

<h2>Stepwise Typing in Practice</h2>

Function `show_count` without type annotations

In [1]:
def show_count(count, word):
    if count == 1:
        return f"1 {word}"
    
    count_str = str(count) if count else 'No'

    return f"{count_str} {word}s"

In [2]:
show_count(1, 'bird')

'1 bird'

In [3]:
show_count(2, 'bird')

'2 birds'

In [4]:
show_count(0, 'bird')

'No birds'

<h3>Start working with <code>mypy</code></h3>

`mypy` is a static type checker for Python that analyzes code for type errors and inconsistencies, helping to catch bugs and improve code quality.

Running the `mypy` command for script with `show_count` function:

`Success: no issues found in 1 source file`

`show_count` function tests without type annotations

In [5]:
from pytest import mark


# Define multiply sets of input data for a test function
@mark.parametrize('qty, expected', [
    (1, '1 part'),
    (2, '2 parts')
])

def test_show_count(qty, expected):
    got = show_count(qty, 'part')
    
    assert got == expected

def test_show_count_zero():
    got = show_count(0, 'part')

    assert got == 'No parts'

Running the `mypy` command for script with `show_count` function tests with the `--disallow-untyped-defs` parameter, which forces `mypy` to mark all function definitions that do not have type annotations:

<code>error: Function is missing a type annotation
error: Function is missing a return type annotation
note: Use "-> None" if function does not return a value
Found 2 errors in 1 file (checked 1 source file)</code>

<h3>Default Parameter Value</h3>

Function `show_count` with optional parameter

In [6]:
def show_count(count: int, singular: str, plural: str = '') -> str:
    if count == 1:
        return f"1 {singular}"

    count_str = str(count) if count else 'No'

    if not plural:
        plural = singular + 's'

    return f"{count_str} {plural}"

Running the `mypy` command for script with `show_count` function:

`Success: no issues found in 1 source file`

<h3><code>None</code> as the Default Value</h3>

`Optional[str]` means that `plural` can be types `str` or `None` 

In [7]:
from typing import Optional


def show_count(count: int, singular: str, plural: Optional[str] = None) -> str:
    if count == 1:
        return f"1 {singular}"

    count_str = str(count) if count else 'No'

    if not plural:
        plural = singular + 's'

    return f"{count_str} {plural}"

<h2>Types suiatable for use in Annotations</h2>

<h3>Type <code>Any</code></h3>

`Any` indicates that a variable can hold a value of any type, and it is often used to denote dynamic or unspecified types within a program.

In [8]:
from typing import Any


def double(x: Any) -> Any:
    return x * 2

<h3><code>Optional</code> and <code>Union</code> Types</h3>

`Optional` indicates that a variable can be of a specific type or `None`, while `Union` allows specifying multiple possible types.

In [9]:
from typing import Union


def ord(c: Union[str, bytes]) -> int:
    pass

def parse_token(token: str) -> Union[str, float]:
    """Function that accepts str, but can return str or float"""
    try:
        return float(token)
    except ValueError:
        return token

Starting from Python 3.10, users can use `str | bytes` instead of `Union[str, bytes]`, offering a shorter alternative without needing to import `Optional` and `Union` from the typing module:

<code>plural: Optional[str] = None  # before
plural: str | None = None  # after</code>

<h3>Tuples Types</h3>

<i>Tuples as records</i>

To accept tuples containing the name of a city, its population and country, the type annotation should look like this: `tuple[str, float, str]`.

In [10]:
def city_info(data: tuple[str, float, str]) -> str:
    return f"The city is {data[0]}, the population is {data[1]} million, the coutry is {data[2]}."

city_info(('New York', 8.46, 'USA'))

'The city is New York, the population is 8.46 million, the coutry is USA.'

<i>Tuples as records with annotated fields</i>

Using a tuple containing many fields in annotations, or specific tuple types that occur in many places in the code.

In [11]:
from typing import NamedTuple


class CityInfo(NamedTuple):
    name: str
    population: float
    country: str

def city_info(data: CityInfo) -> str:
    return f"The city is {data.name}, the population is {data.population}, the country is {data.country}."

city_data = CityInfo(
    name='New York',
    population=8.46,
    country='USA'
)
city_info(city_data)

'The city is New York, the population is 8.46, the country is USA.'

<i>Tuples as immutable Sequences</i>

An indefinite-length tuple used as an immutable list in an annotation must be specified with a single type followed by a comma and three dots. Annotation `stuff: tuple[Any, ...]` is a tuple of identifinte-length containing objects of any type.

In [12]:
from collections.abc import Sequence


def columnize(
    sequence: Sequence[str],
    num_columns: int = 0,
) -> list[tuple[str, ...]]:  # add three dots to accept any number of elemnts
    if num_columns == 0:
        num_columns = round(len(sequence) ** 0.5)

    num_rows, reminder = divmod(len(sequence), num_columns)
    num_rows += bool(reminder)

    return [
        tuple(sequence[i::num_rows])
        for i in range(num_rows)
    ]

In [13]:
animals = 'drake fawn heron ibex koala lynx tahr xerus yak zapus'.split()

for row in columnize(animals):
    print(''.join(f"{word:10}" for word in row))

drake     koala     yak       
fawn      lynx      zapus     
heron     tahr      
ibex      xerus     


<h3>Generalized Mappings</h3>

**Generalized Mappings** are annotated types of the form `MappingType[KeyType, ValueType]`.

In [14]:
import sys
import re
import unicodedata

from collections.abc import Iterator


RE_WORD = re.compile(r'\w+')
STOP_CODE = sys.maxunicode + 1

def tokenize(text: str) -> Iterator[str]:  # generator function
    """Return iterable of uppercased words"""
    for match in RE_WORD.finditer(text):
        yield match.group().upper()

def name_index(start: int = 32, end: int = STOP_CODE) -> dict[str, set[str]]:
    index: dict[str, set[str]] = {}  # local variable annotated

    for char in (chr(i) for i in range(start, end)):
        # Assign the result of unicodedata.name() to variable 'name'
        # if it is not empty, else use an empty string
        
        if name := unicodedata.name(char, ''):
            for word in tokenize(name):
                index.setdefault(word, set()).add(char)

    return index

In [15]:
index = name_index(32, 65)
index['SIGN']

{'#', '$', '%', '+', '<', '=', '>'}

In [16]:
index['DIGIT']

{'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [17]:
index['DIGIT'] & index['EIGHT']

{'8'}

<h3>Type <code>Iterable</code></h3>

The `collections.abc` module provides abstract base classes (ABCs) for containers such as sequences, mappings, and sets. The `Iterable` ABC, specifically, represents classes that can be iterated over with a loop or consumed by functions like `list()` or `tuple()`.

In [18]:
from collections.abc import Iterable


def zip_replace(text: str, changes: Iterable[tuple[str, str]]) -> str:
    for from_, to in changes:
        text = text.replace(from_, to)

    return text

In [19]:
l33t = [
    ('a', '4'),
    ('e', '3'),
    ('i', '1'),
    ('o', '0')
]
text = 'mad skilled noob powned leet'

zip_replace(text, l33t)

'm4d sk1ll3d n00b p0wn3d l33t'